<a href="https://colab.research.google.com/github/Vlasdislav/recsys/blob/main/recsys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Установка и импорт библиотек

In [ ]:
!pip install pymorphy2
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_sm
!python -m spacy download ru_core_news_sm
!pip install jupyter-dash
!pip install dash-bootstrap-components

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 25.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13707 sha256=9c27c8dab28091a8b18751b07917597f320ffde210a264940e83e20955004ff3
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.0 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.7.2
    Uninstalling setuptools-67.7.2:
      Successfully uninstalled setuptools-67.7.2
ERROR: pip's dependency resolver does not currently take into acc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 37.8 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.5.2
    Uninstalling spacy-3.5.2:
      Successfully uninstalled spacy-3.5.2
2023-06-12 19:40:02.437809: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-12 19:40:04.025358: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 68.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load th

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pymorphy2.analyzer import MorphAnalyzer
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering

from ast import literal_eval
from random import sample
import re

from sklearn import metrics
from sklearn.metrics.pairwise import cosine_similarity
import spacy

from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, State, Output

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
<ipython-input-3-5312e76d2e60>:27: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
<ipython-input-3-5312e76d2e60>:28: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


# Подготовка данных

In [ ]:
NUM_DATA = 1000
NUM_CLUSTERS = 23

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/IT_vacancies_full.csv')[:NUM_DATA]  # Получаем первые NUM_DATA вакансий
data_small = data.loc[:, ['Ids', 'Employer', 'Name', 'Description']]                                # Получение нового DataFrame для дальнейшей работы

In [ ]:
data

,Ids,Employer,Name,Salary,From,To,Experience,Schedule,Keys,Description,Area,Professional roles,Specializations,Profarea names,Published at
0,49313809,Space307,Golang Developer (Кипр),True,251322.0,NaN,От 3 до 6 лет,Полный день,"['Docker', 'Golang', 'Redis', 'Английский язык...",Мы в Space307 разрабатываем международную торг...,Санкт-Петербург,"['Программист, разработчик']","['Программирование, Разработка']","['Информационные технологии, интернет, телеком']",2021-12-02 12:15:37+03:00
1,48813842,Монополия,Е-mail маркетолог,True,60900.0,NaN,От 1 года до 3 лет,Полный день,"['Грамотность', 'Написание текстов', 'Грамотна...",С 2015 года наш IT блок меняет рынок автотранс...,Санкт-Петербург,['Менеджер по маркетингу и рекламе'],['Маркетинг'],"['Информационные технологии, интернет, телеком']",2021-12-02 10:33:15+03:00
2,49413720,Eden Springs,Оператор call-центра (удаленно),False,NaN,NaN,От 1 года до 3 лет,Удаленная работа,"['Клиентоориентированность', 'Ориентация на ре...",Что нужно будет делать: Принимать входящие зв...,Санкт-Петербург,"['Оператор call-центра, специалист контактного...","['Маркетинг', 'Продажи по телефону, Телемаркет...","['Продажи', 'Информационные технологии, интерн...",2021-12-02 10:29:37+03:00
3,46460892,Импорт Хоум,Ведущий SMM специалист,True,60000.0,80000.0,От 1 года до 3 лет,Полный день,"['Продвижение бренда', 'Креативность', 'Adobe ...",В данный момент мы ищем в нашу команду самого ...,Санкт-Петербург,"['SMM-менеджер, контент-менеджер']","['Управление маркетингом', 'PR, Маркетинговые ...","['Информационные технологии, интернет, телеком...",2021-12-01 16:57:02+03:00
4,49555567,Pride Games Studio,UX/UI Designer,False,NaN,NaN,От 1 года до 3 лет,Полный день,"['UI', 'UX', 'gamedev', 'game design', 'проект...",Pride Games Studio — это команда единомышленни...,Санкт-Петербург,"['Дизайнер, художник']","['Игровое ПО', 'Программирование, Разработка',...","['Маркетинг, реклама, PR', 'Информационные тех...",2021-12-01 16:48:24+03:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48559,49108967,A-N SECURITY,Монтажник слаботочных систем,True,60000.0,NaN,От 1 года до 3 лет,Полный день,"['Мобильность', 'Работа в команде', 'Системы о...",Обязанности: ведение технической документации...,Москва,"['Сервисный инженер, механик']","['Передача данных и доступ в интернет', 'Сетев...","['Рабочий персонал', 'Информационные технологи...",2021-10-25 11:40:23+03:00
48560,49108929,Филиал ФКУ Налог-Сервис ФНС России в г. Москве,Системный администратор,True,60000.0,NaN,От 1 года до 3 лет,Полный день,"['Windows 7', 'Windows 8', 'Windows 10', 'Unix...",Функциональные обязанности: Сопровождение опе...,Москва,['Системный администратор'],"['Системный администратор', 'Инженер', 'Поддер...","['Информационные технологии, интернет, телеком']",2021-10-25 11:39:41+03:00
48561,47462568,Market Warrior,Менеджер по работе с маркетплейсами (офис),True,50000.0,NaN,От 1 года до 3 лет,Полный день,"['Работа с большим объемом информации', 'Управ...","Мы молодая команда, занимающаяся помощью в раб...",Москва,['Координатор отдела продаж'],['Управление проектами'],"['Информационные технологии, интернет, телеком']",2021-10-25 11:38:52+03:00
48562,48713091,Кон.Трол,"Менеджер по продажам (Серверное оборудование, ...",True,120000.0,300000.0,От 1 года до 3 лет,Полный день,"['B2B Продажи', 'Ведение переговоров', 'CRM', ...",GAGAR&gt;IN ( ГК &quot;Con.Troll&quot; ) - рос...,Москва,"['Менеджер по продажам, менеджер по работе с к...","['Телекоммуникации, Сетевые решения', 'Услуги ...","['Информационные технологии, интернет, телеком...",2021-10-25 11:38:34+03:00


In [ ]:
def to_text(arr):
  """Конвертация списка слов в текст"""
  return arr.apply(lambda x: ' '.join(literal_eval(x))
    if isinstance(x, str) else '')

def create_combined_data(x):
  """Объединение значимых признаков в один текст"""
  return x['Name'] + ' ' + to_text(x['Professional roles']) + ' ' + to_text(x['Keys']) + ' ' + to_text(x['Specializations']) + ' ' + to_text(x['Profarea names'])

data_small['Description'] = create_combined_data(data)  # Перезаписываем признак Description новым описанием

# Очистка данных и векторизация

In [ ]:
STOPWORDS = set(stopwords.words('russian')) | set(stopwords.words('english'))   # Множество русских и английских стоп слов
PATTERN_ALL_SIGNS = '[«»—.,!@#$%^&*()_+=-?:;№"\|/<>{}-]'                        # Шаблон знаков

def del_all_signs(text):
  """Удаление всех знаков из текста"""
  return re.sub(PATTERN_ALL_SIGNS, ' ', text)

def del_stop_words(text):
  """Удаление стоп слова из текста"""
  text_split = text.split(' ')
  for word in text_split:
    if word in STOPWORDS:
      text_split.remove(word)
  return ' '.join(text_split)

morph = MorphAnalyzer()

def text2normal(text):
  """Лемматизация всех слов текста"""
  text = text.split()
  res = []
  for word in text:
    res.append(morph.parse(word)[0].normal_form)
  return ' '.join(res)

def text_clean(text):
  """Основная функция очистки текста,
    исполняет ранее описанные функции"""
  text = text.lower()
  text = del_all_signs(text)
  text = text2normal(text)
  text = del_stop_words(text)
  return text

data_small['description_normal'] = data_small['Description'].apply(text_clean)  # Получаем текст пригодный для векторизации

In [ ]:
documents = data_small['description_normal'].values.astype('U') # Получаем данные из ранее подготовленных текстов
vectorizer = TfidfVectorizer(stop_words=['english', 'russian']) # Инициализируем объект для векторизации текста
features = vectorizer.fit_transform(documents)                  # Применение TfidfVectorizer, чтобы преобразовать текст в векторы признаков

# Определим количество кластеров

In [ ]:
scores = []

for i in range(2, 41):
  model = KMeans(n_clusters=i, init='k-means++', max_iter=100, n_init=100).fit(features)
  score = metrics.silhouette_score(features, model.labels_, metric='euclidean', sample_size=features.shape[0])
  scores.append(score)

num_clusters = np.argmax(scores)
print('\nОптимальное количество кластеров =', num_clusters)

KeyboardInterrupt: ignored

Exception ignored in: 'sklearn.cluster._k_means_common._relocate_empty_clusters_sparse'
Traceback (most recent call last):
  File "<__array_function__ internals>", line 177, in where
KeyboardInterrupt: 


KeyboardInterrupt: ignored

Exception ignored in: 'sklearn.cluster._k_means_common._relocate_empty_clusters_sparse'
Traceback (most recent call last):
  File "<__array_function__ internals>", line 177, in where
KeyboardInterrupt: 


# Кластеризация методом k-means

In [ ]:
model = KMeans(n_clusters=NUM_CLUSTERS, init='k-means++', max_iter=100, n_init=100).fit(features)
data_small['cluster'] = model.labels_

In [ ]:
with open("input.txt") as f:
  s = f.read()
  print(s)

In [ ]:
print('Cluster centroids: \n')
older_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names_out()

content_clusters = []

for i in range(NUM_CLUSTERS):
  print('Cluster %d:' % i)
  content_clusters.append([terms[j] for j in older_centroids[i, :10]])
  for j in older_centroids[i, :10]:
    print(' %s' % terms[j])
  print('---------------')

# Результат web (кластеризация)

In [ ]:
def predict_top(id, data_small, top):
  """Функция возвращает для конкретной вакансии по id топ подходящих вакансий в количестве top (Кластеризация k-means)"""
  tops = [x for x in range(len(data_small['cluster'])) if x != id and data_small['cluster'][x] == data_small['cluster'][id]]
  # return data.iloc[sample(tops, len(tops))[:top]] # Возвращает топ случайных вакансий из кластера
  return data.iloc[tops[:top]]                      # Возвращает топ ближайших вакансий

In [ ]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

search_bar = dbc.Row(
    [
        dbc.Col(dbc.Input(id='input-on-submit', placeholder="Введите индекс вакансии", type="search")),
        dbc.Col(
            dbc.Button(
                "Поиск", id='submit-val', color="primary", className="ms-2", n_clicks=0
            ),
            width="auto",
        ),
    ],
    className="g-0 ms-auto flex-nowrap mt-3 mt-md-0",
    align="center",
)

app.layout = html.Div([
    html.Br(),
    dbc.Row([
        dbc.Col(html.Div(), md=3),
        dbc.Col(
            search_bar,
            md=6
        ),
        dbc.Col(html.Div(), md=3),
    ]),
    html.Br(),
    html.Div(id='container-button-basic'),
])

@app.callback(
    Output('container-button-basic', 'children'),
    Input('submit-val', 'n_clicks'),
    State('input-on-submit', 'value')
)

def update_output(n_clicks, value):
    top = 3

    prediction = predict_top(int(value), data_small, top)

    output = []

    output.append(
        html.Div(
          dbc.Container(
              [
                  html.H1("#" + str(value) + " " + data['Name'][int(value)], className="display-6"),
                  html.P(
                     data['Employer'][int(value)],
                      className="lead",
                  ),
                  html.Hr(className="my-2"),
                  html.P(
                      data['Description'][int(value)],
                      style={ "overflow": "hidden",  "-webkit-line-clamp": "5", "-webkit-box-orient": "vertical", "display": "-webkit-box" }
                  ),
              ],
              fluid=True,
              className="py-3",
          ),
          className="p-3 bg-light rounded-3",
        style={ "width": "1200px", "margin": "auto" }
        )
      )
    output.append(html.Br())

    for i in range(top):
      output.append(
        html.Div(
          dbc.Container(
              [
                  html.H1("#" + str(prediction.index[i]) + " " + prediction.iloc[i]['Name'], className="display-6"),
                  html.P(
                      prediction.iloc[i]['Employer'],
                      className="lead",
                  ),
                  html.Hr(className="my-2"),
                  html.P(
                      prediction.iloc[i]['Description'],
                      style={ "overflow": "hidden",  "-webkit-line-clamp": "5", "-webkit-box-orient": "vertical", "display": "-webkit-box" }
                  ),
              ],
              fluid=True,
              className="py-3",
          ),
          className="p-3 bg-light rounded-3",
          style={ "width": "1200px", "margin": "auto" }
        )
      )
      output.append(html.Br())

    return html.P(output)

app.run_server(host='127.0.0.1')

Dash is running on http://127.0.0.1:8050/



INFO:dash.dash:Dash is running on http://127.0.0.1:8050/



Dash app running on:


<IPython.core.display.Javascript object>

# Косинусное растояние

In [ ]:
def extract_best_indices(m, topk, mask=None):
    if len(m.shape) > 1:
        cos_sim = np.cosine_similarity(m, axis=0)
    else:
        cos_sim = m

    index = np.argsort(cos_sim)[::-1]
    if mask is not None:
        assert mask.shape == m.shape
        mask = mask[index]
    else:
        mask = np.ones(len(cos_sim))
    mask = np.logical_or(cos_sim[index] != 0, mask)
    best_index = index[mask][:topk]
    return best_index

In [ ]:
def predict_spacy(model, query_sentence, embed_mat, topk):
    topk += 1
    query_embed = model(query_sentence)
    mat = np.array([query_embed.similarity(line) for line in embed_mat])

    mat_mask = np.array(
        [True if line.vector_norm else False for line in embed_mat])
    best_index = extract_best_indices(mat, topk=topk, mask=mat_mask)
    return best_index[1:topk]

In [ ]:
nlp = spacy.load("ru_core_news_sm")
data_small['spacy_description'] = data_small['Description'].apply(nlp)
embed_mat = data_small['spacy_description'].values

In [ ]:
def predict_top(id, data_small, top):
    """Функция возвращает для конкретной вакансии по id
    топ подходящих вакансий в количестве top (Косинусная схожесть)"""
    test_sentence = data_small.iloc[id]['Description']
    embed_mat = data_small['spacy_description'].values
    best_index = predict_spacy(nlp, test_sentence, embed_mat, top)
    return data.iloc[best_index]

In [ ]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

search_bar = dbc.Row(
    [
        dbc.Col(dbc.Input(id='input-on-submit', placeholder="Введите индекс вакансии", type="search")),
        dbc.Col(
            dbc.Button(
                "Поиск", id='submit-val', color="primary", className="ms-2", n_clicks=0
            ),
            width="auto",
        ),
    ],
    className="g-0 ms-auto flex-nowrap mt-3 mt-md-0",
    align="center",
)

app.layout = html.Div([
    html.Br(),
    dbc.Row([
        dbc.Col(html.Div(), md=3),
        dbc.Col(
            search_bar,
            md=6
        ),
        dbc.Col(html.Div(), md=3),
    ]),
    html.Br(),
    html.Div(id='container-button-basic'),
])

@app.callback(
    Output('container-button-basic', 'children'),
    Input('submit-val', 'n_clicks'),
    State('input-on-submit', 'value')
)

def update_output(n_clicks, value):
    top = 3

    prediction = predict_top(int(value), data_small, top)

    output = []

    output.append(
        html.Div(
          dbc.Container(
              [
                  html.H1("#" + str(value) + " " + data['Name'][int(value)], className="display-6"),
                  html.P(
                     data['Employer'][int(value)],
                      className="lead",
                  ),
                  html.Hr(className="my-2"),
                  html.P(
                      data['Description'][int(value)],
                      style={ "overflow": "hidden",  "-webkit-line-clamp": "5", "-webkit-box-orient": "vertical", "display": "-webkit-box" }
                  ),
              ],
              fluid=True,
              className="py-3",
          ),
          className="p-3 bg-light rounded-3",
        style={ "width": "1200px", "margin": "auto" }
        )
      )
    output.append(html.Br())

    for i in range(top):
      output.append(
        html.Div(
          dbc.Container(
              [
                  html.H1("#" + str(prediction.index[i]) + " " + prediction.iloc[i]['Name'], className="display-6"),
                  html.P(
                      prediction.iloc[i]['Employer'],
                      className="lead",
                  ),
                  html.Hr(className="my-2"),
                  html.P(
                      prediction.iloc[i]['Description'],
                      style={ "overflow": "hidden",  "-webkit-line-clamp": "5", "-webkit-box-orient": "vertical", "display": "-webkit-box" }
                  ),
              ],
              fluid=True,
              className="py-3",
          ),
          className="p-3 bg-light rounded-3",
          style={ "width": "1200px", "margin": "auto" }
        )
      )
      output.append(html.Br())

    return html.P(output)

app.run_server(host='0.0.0.0')

Dash is running on http://0.0.0.0:8050/



INFO:dash.dash:Dash is running on http://0.0.0.0:8050/



Dash app running on:


<IPython.core.display.Javascript object>